# 라이브러리 로드

In [1]:
import pandas as pd
import numpy as np


In [2]:
path = "/content/drive/MyDrive/Colab Notebooks/DataScience/Data/cafeteria/"
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

# 데이터 확인

In [11]:
train.shape, test.shape

((1205, 12), (50, 10))

In [3]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [4]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [5]:
submission.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


# 요일을 숫자로 매핑

In [6]:
train['요일'] = train['요일'].map({"월" : 0, "화" : 1, "수" : 2, "목" : 3, "금" : 4})
test['요일'] = test['요일'].map({"월" : 0, "화" : 1, "수" : 2, "목" : 3, "금" : 4})

# 수치 데이터만 피쳐로 선택

In [15]:
X = train[['요일', "본사정원수","본사출장자수","본사시간외근무명령서승인건수","현본사소속재택근무자수"]]
y1 = train['중식계']
y2 = train['석식계']

X_test = test[['요일', "본사정원수","본사출장자수","본사시간외근무명령서승인건수","현본사소속재택근무자수"]]

In [16]:
from sklearn.model_selection import train_test_split

X1_train, X1_val, y1_train , y1_val = train_test_split(X, y1, test_size = 0.25, random_state=42)
X2_train, X2_val, y2_train , y2_val = train_test_split(X, y2, test_size = 0.25, random_state=42)

In [17]:
print("X1 : ")
print(X1_train.shape, y1_train.shape)
print(X1_val.shape, y1_val.shape)

X1 : 
(903, 5) (903,)
(302, 5) (302,)


In [18]:
print("X1 : ")
print(X2_train.shape, y2_train.shape)
print(X2_val.shape, y2_val.shape)

X1 : 
(903, 5) (903,)
(302, 5) (302,)


# 점심과 저녁 식수 인원을 예측할 2개 모델 생성

In [27]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

rf_model1 = RandomForestRegressor(n_jobs=-1, random_state=42)
rf_model2 = RandomForestRegressor(n_jobs=-1, random_state=42)
gb_model1 = GradientBoostingRegressor(n_estimators=300,random_state=42)
gb_model2 = GradientBoostingRegressor(n_estimators=300,random_state=42)
xgb_model1 = XGBRegressor(n_estimators=300, n_jobs=-1, random_state=42)
xgb_model2 = XGBRegressor(n_estimators=300, n_jobs=-1, random_state=42)
lgbm_model1 = LGBMRegressor(random_state=42, n_jobs=-1)
lgbm_model2 = LGBMRegressor(random_state=42, n_jobs=-1)

In [39]:
models1 = [rf_model1, gb_model1, xgb_model1,lgbm_model1]
models2 = [rf_model2, gb_model2,xgb_model2, lgbm_model2]
for model1 in models1:
  model1.fit(X1_train, y1_train)
  y_pred1 = model1.predict(X1_val)
  print(model1.__class__.__name__)
  print("중식계 : {}".format(MAE(y1_val, y_pred1)))
for model2 in models2:
  model2.fit(X2_train, y2_train)
  y_pred2 = model2.predict(X2_val)
  print(model2.__class__.__name__)
  print("석식계 : {}".format(MAE(y2_val, y_pred2)))

RandomForestRegressor
중식계 : 84.37913907284768
GradientBoostingRegressor
중식계 : 79.6353750178602
[06:26:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor
중식계 : 80.14081770536916
LGBMRegressor
중식계 : 82.59534771090912
RandomForestRegressor
석식계 : 68.39897350993377
GradientBoostingRegressor
석식계 : 69.52450891191219
[06:26:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor
석식계 : 69.31192835119387
LGBMRegressor
석식계 : 69.16770216081987


# 학습 및 추론

In [38]:
gb_model1 = GradientBoostingRegressor(n_estimators=300,random_state=42)
gb_model2 = GradientBoostingRegressor(n_estimators=300,random_state=42)

In [35]:
gb_model1.fit(X1_train, y1_train)
gb_model2.fit(X2_train, y2_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=300,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [36]:
y1_pred = gb_model1.predict(X1_val)
y2_pred = gb_model2.predict(X2_val)

In [37]:
from sklearn.metrics import mean_absolute_error as MAE

print("중식계 : ",MAE(y1_val, y1_pred))
print("석식계 : ",MAE(y2_val, y2_pred))

중식계 :  79.6353750178602
석식계 :  69.52450891191219


In [43]:
gb_model1 = GradientBoostingRegressor(n_estimators=300,random_state=42)
gb_model2 = GradientBoostingRegressor(n_estimators=300,random_state=42)

gb_model1.fit(X, y1)
gb_model2.fit(X, y2)

y1_pred = gb_model1.predict(X_test)
y2_pred = gb_model2.predict(X_test)

In [58]:
rf_model1 = RandomForestRegressor(n_jobs=-1, random_state=42)
rf_model2 = RandomForestRegressor(n_jobs=-1, random_state=42)

rf_model1.fit(X, y1)
rf_model2.fit(X, y2)

y1_pred = rf_model1.predict(X_test)
y2_pred = rf_model2.predict(X_test)

In [59]:
submission['중식계'] = y1_pred
submission['석식계'] = y2_pred

In [60]:
submission.head()

,일자,중식계,석식계
0,2021-01-27,1058.21,366.02
1,2021-01-28,967.91,384.52
2,2021-01-29,596.09,254.61
3,2021-02-01,1184.24,449.98
4,2021-02-02,973.61,413.13


In [61]:
sub_path = "/content/drive/MyDrive/Colab Notebooks/DataScience/My CSV/Dacon/cafeteria/"
sub_name = "baseline_submission_real.csv"
submission.to_csv(sub_path + sub_name, index=False)